In [ ]:
%flink.ssql

SELECT ticker, `date`, close_price, t.avg_close_price, t.ema_close_price
FROM (
	SELECT ticker, `date`, close_price, 
	AVG(close_price) OVER (
		PARTITION BY ticker
		ORDER BY event_time
		ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
	) AS avg_close_price, 
	EXP((SUM(LN(close_price)) OVER (
		PARTITION BY ticker
		ORDER BY event_time
		ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
	) / 10 )) AS ema_close_price
	FROM stock_table
) t
WHERE close_price < t.ema_close_price